In [ ]:
import pandas as pd
from pathlib import Path
from IPython.display import display, Markdown, HTML
import json
from jinja2 import Template
import warnings
warnings.filterwarnings('ignore')

from itables import to_html_datatable as DT
import altair as alt

report_dir = Path("../")

In [ ]:
antismash_table = report_dir / "tables/df_antismash_6.1.1_summary.csv"
gtdb_table = report_dir / "tables/df_gtdb_meta.csv"

df_antismash = pd.read_csv(antismash_table, index_col=0)
df_gtdb = pd.read_csv(gtdb_table, index_col=0)

df_raw = pd.concat([df_antismash, df_gtdb], axis=1)
df = df_raw.loc[:, ["genome_id", "source", 'Organism', "strain", 'bgcs_count', 'bgcs_on_contig_edge']]
for i in df.index:
    gid = df.loc[i, 'genome_id']
    server_path = "<a href='http://localhost:{{ project().file_server() }}/antismash/6.1.1/"
    df.loc[i, "genome_id"] = server_path + f"{gid}/'>{gid}</a>"
df = df.reset_index(drop=True)

# AntiSMASH Report
Summary of AntiSMASH results from {{ project().name }}

In [ ]:
source = df_raw

chart = alt.Chart(source).mark_circle().encode(
    x = 'bgcs_count',
    y = 'bgcs_on_contig_edge',
    color='Genus',
    tooltip=['genome_id', 'bgcs_count', 'bgcs_on_contig_edge', 'protoclusters_count', 'cand_clusters_count']
).properties(
    width=400,
    height=400,
    title = "BGC distribution overview",
).interactive()

chart = chart.configure_title(fontSize=20, offset=10, orient='top', anchor='middle')

chart

## Summary Table

In [ ]:
display(HTML(DT(df)))

## References
{% for i in project().rule_used['antismash']['references'] %}
- {{ i }}
{% endfor %}